# 새로운 노래
* 전 주에 없던 새로운 노래 확인

## 라이브러리

In [1]:
import pandas as pd
import os

## 1주일 동안 새로 진입한 곡 평균

In [200]:
f_path = 'c:/data/Wantreez/Crawling/music'
try:
    os.chdir(f_path)
except:
    pass

In [201]:
code_path = os.getcwd().replace('\\', '/')
code_path

'c:/data/Wantreez/Crawling/music'

In [202]:
# 전 주 파일 날짜(일주일)
start_date = '2022-04-25'
end_date = '2022-05-01'

In [203]:
# 비교할 파일 날짜(일주일)
new_start_date = '2022-05-02'
new_end_date = '2022-05-08'
new_dates_df = pd.date_range(new_start_date, new_end_date).astype(str)

In [204]:
# 음원 사이트
site_list = ['flo', 'bugs', 'genie', 'melon', 'vibe']
for site_name in site_list:
    try:
        folder_path = f'/crawled_data/mod_{site_name}/'
        path = code_path + folder_path
        try:
            os.chdir(path)
        except:
            pass

        # 파일 이름들
        file_list = os.listdir()

        # 전체 df 생성
        total_df = pd.DataFrame()
        for one in file_list:
            temp = pd.read_excel(one)
            total_df = pd.concat([total_df, temp], ignore_index=True)
        total_df['날짜'] = total_df['날짜'].astype(str)

        # 전주, 이번주 df
        before_df = total_df[ (total_df['날짜'] >= start_date) & (total_df['날짜'] <= end_date)].reset_index(drop=True)
        new_df = total_df[ (total_df['날짜'] >= new_start_date) & (total_df['날짜'] <= new_end_date)].reset_index(drop=True)

        # 전주 노래 목록
        title_list = before_df['곡'].unique()
        # 이번주 새로 진입한 곡 df
        pop_title_list = []
        pop_artist_list = []
        pop_date_df = pd.DataFrame()
        for new_one_date in new_dates_df:
            new_date_df = total_df[total_df['날짜'] == new_one_date].reset_index(drop=True)

            for i in range(new_date_df.shape[0]):
                one_title = new_date_df['곡'][i]
                one_artist = new_date_df['가수'][i]
                if one_title not in title_list:
                    pop_title_list.append(one_title)
                    pop_artist_list.append(one_artist)

            for two_title, two_artist in zip(pop_title_list, pop_artist_list):
                temp = new_df[ (new_df['곡'] == two_title) & (new_df['가수'] == two_artist)]
                pop_date_df = pd.concat([pop_date_df, temp], ignore_index=True)
                pop_date_df = pop_date_df.drop_duplicates(['날짜', '곡', '가수'], ignore_index=True)

        # 새로 진입한 곡의 평균
        mean_pop_df = pop_date_df.groupby(['소속사', '곡', '가수']).agg('mean').reset_index()
        mean_pop_df.columns = ['소속사', '곡', '가수', '순위평균']
        mean_pop_df['순위평균'] = round(mean_pop_df['순위평균'], 1)

        # 파일 저장
        save_folder = f'/crawled_data/outlier_data/{new_start_date}_{new_end_date}/{site_name}/'
        path = code_path + save_folder
        file_name = f'[{site_name}]{new_start_date}_{new_end_date}_pop_df.xlsx'
        mean_pop_df.to_excel(path + file_name, index=False, encoding='utf-8')
        print(f'{file_name} 출력완료.')
    except:
        pass

[bugs]2022-05-02_2022-05-08_pop_df.xlsx 출력완료.
[genie]2022-05-02_2022-05-08_pop_df.xlsx 출력완료.
[melon]2022-05-02_2022-05-08_pop_df.xlsx 출력완료.
[vibe]2022-05-02_2022-05-08_pop_df.xlsx 출력완료.


## 1주일 동안 새로 진입한 곡 중 10위권 곡만 추출

In [206]:
# 음원 사이트
site_list = ['flo', 'bugs', 'genie', 'melon', 'vibe']
for site_name in site_list:
    folder_path = f'/crawled_data/mod_{site_name}/'
    path = code_path + folder_path
    try:
        os.chdir(path)
    except:
        pass

    # 파일 이름들
    file_list = os.listdir()

    # 전체 df 생성
    total_df = pd.DataFrame()
    for one in file_list:
        temp = pd.read_excel(one)
        total_df = pd.concat([total_df, temp], ignore_index=True)
    total_df['날짜'] = total_df['날짜'].astype(str)

    # 전주, 이번주 df
    before_df = total_df[ (total_df['날짜'] >= start_date) & (total_df['날짜'] <= end_date)].reset_index(drop=True)
    new_df = total_df[ (total_df['날짜'] >= new_start_date) & (total_df['날짜'] <= new_end_date)].reset_index(drop=True)

    # 전주, 이번주 Top10 df
    before_top10_df = before_df[before_df['순위'] <= 10]
    current_top10_df = new_df[new_df['순위'] <= 10]
    # 전주 Top10 음원들의 순위 평균
    mean_before_top10_df = before_top10_df.groupby(['사이트', '곡', '가수']).agg('mean').reset_index()
    mean_before_top10_df['순위'] = round(mean_before_top10_df['순위'], 1)
    mean_before_top10_df.columns = ['사이트', '곡', '가수', '과거순위평균']

    # 이번주 Top10 음원들의 순위 평균
    mean_current_top10_df = current_top10_df.groupby(['사이트', '곡', '가수']).agg('mean').reset_index()
    mean_current_top10_df['순위'] = round(mean_current_top10_df['순위'], 1)
    mean_current_top10_df.columns = ['사이트', '곡', '가수', '현재순위평균']

    # 전주, 이번주 Top10 df 병합
    bvsc_df = pd.merge(mean_before_top10_df, mean_current_top10_df, on=['곡', '가수'], how='inner')
    bvsc_df = bvsc_df.drop('사이트_y', axis=1)
    # 전주 대비 상승폭
    bvsc_df['전주 대비 상승폭'] = bvsc_df['현재순위평균'] - bvsc_df['과거순위평균']
    # 파일 저장
    save_folder = f'/crawled_data/outlier_data/{new_start_date}_{new_end_date}/{site_name}/'
    path = code_path + save_folder
    file_name = f'[{site_name}]{new_start_date}_{new_end_date}_Top10_rank_change_df.xlsx'
    bvsc_df.to_excel(path + file_name, index=False, encoding='utf-8')
    print(f'{file_name} 출력완료.')


ValueError: Length mismatch: Expected axis has 5 elements, new values have 4 elements

In [ ]:
pop_title_list = []
pop_artist_list = []
pop_date_df = pd.DataFrame()
for new_one_date in new_dates_df:
    new_date_df = total_df[total_df['날짜'] == new_one_date].reset_index(drop=True)
    new_title_list = new_date_df['곡'].unique()

    for one_title in new_title_list:
        if one_title not in title_list:
            pop_title_list.append(one_title)
            pop_artist_list.append(new_df[new_df['곡'] == one_title]['가수'].values[0]) # 문제 : 같은 곡명, 다른 가수는 구분이 어려움
    print(new_one_date)
    print( len(pop_title_list), len(pop_artist_list) )

    for two_title, two_artist in zip(pop_title_list, pop_artist_list):
        temp = new_df[ (new_df['곡'] == two_title) & (new_df['가수'] == two_artist)]
        pop_date_df = pd.concat([pop_date_df, temp], ignore_index=True)
        pop_date_df = pop_date_df.drop_duplicates(['날짜', '곡', '가수'], ignore_index=True)

2022-05-02
2 2
2022-05-03
4 4
2022-05-04
9 9
2022-05-05
14 14
2022-05-06
19 19
2022-05-07
24 24
2022-05-08
29 29


In [ ]:
pop_date_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   사이트     29 non-null     object
 1   날짜      29 non-null     object
 2   순위      29 non-null     int64 
 3   곡       29 non-null     object
 4   가수      29 non-null     object
 5   앨범      29 non-null     object
 6   소속사     29 non-null     object
 7   소속사_분류  29 non-null     object
dtypes: int64(1), object(7)
memory usage: 1.9+ KB


In [ ]:
pop_date_df

,사이트,날짜,순위,곡,가수,앨범,소속사,소속사_분류
0,vibe,2022-05-02,97,Whisky on the Rock,김연지(씨야),우리들의 블루스 OST Part 1,개인,기타
1,vibe,2022-05-03,74,Whisky on the Rock,김연지(씨야),우리들의 블루스 OST Part 1,개인,기타
2,vibe,2022-05-04,78,Whisky on the Rock,김연지(씨야),우리들의 블루스 OST Part 1,개인,기타
3,vibe,2022-05-05,94,Whisky on the Rock,김연지(씨야),우리들의 블루스 OST Part 1,개인,기타
4,vibe,2022-05-02,98,D-DAY,휘인(Whee In),D-DAY,THE L1VE,기타
5,vibe,2022-05-03,66,D-DAY,휘인(Whee In),D-DAY,THE L1VE,기타
6,vibe,2022-05-04,84,D-DAY,휘인(Whee In),D-DAY,THE L1VE,기타
7,vibe,2022-05-05,100,D-DAY,휘인(Whee In),D-DAY,THE L1VE,기타
8,vibe,2022-05-04,60,FEARLESS,LE SSERAFIM (르세라핌),FEARLESS,SOURCE MUSIC,기타
9,vibe,2022-05-05,46,FEARLESS,LE SSERAFIM (르세라핌),FEARLESS,SOURCE MUSIC,기타


### 새로 진입한 곡의 평균 순위

In [ ]:
pop_date_df.groupby(['소속사', '곡', '가수']).agg('mean')

순위
소속사          곡                  가수                           
P NATION     감동이야 (feat. 성시경)   싸이 (PSY)            88.666667
             이제는 (feat. 화사)     싸이 (PSY)            80.666667
SOURCE MUSIC FEARLESS           LE SSERAFIM (르세라핌)  44.400000
THE L1VE     D-DAY              휘인(Whee In)         87.000000
개인           Whisky on the Rock 김연지(씨야)             85.750000
매직스트로베리 사운드  봄 to 러브            10CM                81.200000
물고기뮤직        다시 만날 수 있을까        임영웅                 56.600000

In [ ]:
mean_pop_df = pop_date_df.groupby(['소속사', '곡', '가수']).agg('mean').reset_index()
mean_pop_df

,소속사,곡,가수,순위
0,P NATION,감동이야 (feat. 성시경),싸이 (PSY),88.666667
1,P NATION,이제는 (feat. 화사),싸이 (PSY),80.666667
2,SOURCE MUSIC,FEARLESS,LE SSERAFIM (르세라핌),44.400000
3,THE L1VE,D-DAY,휘인(Whee In),87.000000
4,개인,Whisky on the Rock,김연지(씨야),85.750000
5,매직스트로베리 사운드,봄 to 러브,10CM,81.200000
6,물고기뮤직,다시 만날 수 있을까,임영웅,56.600000


In [ ]:
mean_pop_df.columns = ['소속사', '곡', '가수', '순위평균']
mean_pop_df

,소속사,곡,가수,순위평균
0,P NATION,감동이야 (feat. 성시경),싸이 (PSY),88.666667
1,P NATION,이제는 (feat. 화사),싸이 (PSY),80.666667
2,SOURCE MUSIC,FEARLESS,LE SSERAFIM (르세라핌),44.400000
3,THE L1VE,D-DAY,휘인(Whee In),87.000000
4,개인,Whisky on the Rock,김연지(씨야),85.750000
5,매직스트로베리 사운드,봄 to 러브,10CM,81.200000
6,물고기뮤직,다시 만날 수 있을까,임영웅,56.600000


In [ ]:
mean_pop_df['순위평균'] = round(mean_pop_df['순위평균'], 1)
mean_pop_df

,소속사,곡,가수,순위평균
0,P NATION,감동이야 (feat. 성시경),싸이 (PSY),88.7
1,P NATION,이제는 (feat. 화사),싸이 (PSY),80.7
2,SOURCE MUSIC,FEARLESS,LE SSERAFIM (르세라핌),44.4
3,THE L1VE,D-DAY,휘인(Whee In),87.0
4,개인,Whisky on the Rock,김연지(씨야),85.8
5,매직스트로베리 사운드,봄 to 러브,10CM,81.2
6,물고기뮤직,다시 만날 수 있을까,임영웅,56.6


### 파일 저장

In [ ]:
code_path

'c:/data/Wantreez/Crawling/music'

In [ ]:
# 파일 저장
save_folder = '/crawled_data/outlier_data/'
path = code_path + save_folder
mean_pop_df.to_excel(path + f'[{site_name}]{new_start_date}_{new_end_date}_pop_df.xlsx', index=False, encoding='utf-8')

# 코드 마지막